# Dealing with missing values in the input dataset

Often times, the input dataset is a coherent set of full entries. That is, given a dataset schema with columns `("x", "y", "z")`, all files in the chain always contain those columns and every entry will be whole. This can be extended to horizontal compositions, called "friends" in the TTree jargon, where the schema of the main dataset can be extended with the columns of the friend(s) dataset(s).

In some cases, entries retrieved in the event loop might be not completely filled. Instead they might present some missing values. In this notebook we explore two notable scenarios where this might happen.

## Scenario 1: Some columns are missing at one or more files in the chain

When processing a dataset made of multiple files, the schema is implicitly assumed to be the one of the first file opened. In some cases, a column present in the first file might not be present in other files, and vice versa. This could happen for example when processing different samples (e.g. data and simulation).

In the next cell, we create a dataset made of three files. The first file contains both columns `x` and `y`, the second file only contains column `y`, the third file only contains column `x`. In practice this would lead to a dataset looking like:

| File # | x | y  |
|--------|---|----|
| 1      | 1 | 11 |
| 1      | 2 | 22 |
| 1      | 3 | 33 |
| 2      |   | 44 |
| 2      |   | 55 |
| 2      |   | 66 |
| 3      | 7 |    |
| 3      | 8 |    |
| 3      | 9 |    |

In [ ]:
from missing_values_utils import DatasetMissingBranches
DatasetMissingBranches.create_dataset()

We create the input dataset for the RDataFrame by chaining the three files together in a TChain object. Operations in the event loop expect the value to be there to function properly, so RDataFrame offers some facilities to decide what to do in case of a missing value:

* `DefaultValueFor(colname, defaultval)`: lets the user provide one default value for the current entry of the input column, in case the value is missing.
* `FilterAvailable(colname)`: works in the same way as the traditional `Filter` operation, where the "expression" is "is the value available?". If so, the entry is kept, if not, it is discarded.
* `FilterMissing(colname)`: works in the same way as the traditional `Filter` operation, where the "expression" is "is the value missing?". If so, the entry is kept, if not, it is discarded.

In [ ]:
import ROOT

chain = ROOT.TChain()
for fname, tname in zip(DatasetMissingBranches.filenames, DatasetMissingBranches.treenames):
    chain.Add(fname + "?#" + tname)

df = ROOT.RDataFrame(chain)

If we want to get a complete view of all columns and all entries, we must make sure that there is always a value to display, so we use `DefaultValueFor` on both columns.

In [ ]:
default_value = ROOT.std.numeric_limits[int].min()

# Example 1: provide a default value for all missing branches
display_1 = (
    df.DefaultValueFor("x", default_value)
    .DefaultValueFor("y", default_value)
    .Display(columnList=("x", "y"), nRows=15)
)
display_1.Print()

If instead we want to use the information that some value is missing, we can for example skip the events where values of column `x` are missing with `FilterAvailable`.

In [ ]:
# Example 2: provide a default value for branch y, but skip events where
# branch x is missing
display_2 = (
    df.DefaultValueFor("y", default_value)
    .FilterAvailable("x")
    .Display(columnList=("x", "y"), nRows=15)
)
display_2.Print()

Finally, we could decide to keep all the events where the column `y` is missing with `FilterMissing`.

In [ ]:
# Example 3: only keep events where branch y is missing and display values for branch x
display_3 = df.FilterMissing("y").Display(columnList=("x",), nRows=15)
display_3.Print()

In [ ]:
DatasetMissingBranches.cleanup_dataset()

## Scenario 2: Some values are missing due to a mismatch when joining with another dataset

The second scenario can happen with unaligned horizontal compositions, i.e. when joining the main dataset with one or more auxiliary datasets and the event order is not guaranteed across them. In case the current event being processed does not match one (or more) of the friend datasets, that would lead to a missing value in the friend column(s).

Let's take for example a dataset with one main file and two auxiliary files. The column `idx` is present in all files and is used to build an index that is then probed during the join step. The main file also has column `x`, the first auxiliary file also has column `y`, the second auxiliary file also has column `z`. Some values for the column `idx` in the main file are not present in the auxiliary files, so that after joining the full view on the dataset would be:

| idx | x | y | z |
|-----|---|---|---|
| 1   | 1 | 4 | 6 |
| 2   | 2 | 5 |   |
| 3   | 3 |   | 7 |

In [ ]:
from missing_values_utils import DatasetMismatchedJoin
DatasetMismatchedJoin.create_dataset()

Now we create the input dataset for the RDataFrame. The indexes for the two auxiliary datasets are built via the `BuildIndex` method. Then the auxiliary datasets are joined with the main one via the `AddFriend` method of the main dataset.

In [ ]:

main_chain = ROOT.TChain(DatasetMismatchedJoin.main_tree_name)
main_chain.Add(DatasetMismatchedJoin.main_file)

aux_chain_1 = ROOT.TChain(DatasetMismatchedJoin.aux_tree_name_1)
aux_chain_1.Add(DatasetMismatchedJoin.aux_file_1)
aux_chain_1.BuildIndex("idx")

aux_chain_2 = ROOT.TChain(DatasetMismatchedJoin.aux_tree_name_2)
aux_chain_2.Add(DatasetMismatchedJoin.aux_file_2)
aux_chain_2.BuildIndex("idx")

main_chain.AddFriend(aux_chain_1)
main_chain.AddFriend(aux_chain_2)

df = ROOT.RDataFrame(main_chain)

As above, we can decide to get a dataset with whole entries by providing a default value in case there is a mismatch in the join and thus a missing value.

In [ ]:
aux_tree_1_colidx = DatasetMismatchedJoin.aux_tree_name_1 + ".idx"
aux_tree_1_coly = DatasetMismatchedJoin.aux_tree_name_1 + ".y"
aux_tree_2_colidx = DatasetMismatchedJoin.aux_tree_name_2 + ".idx"
aux_tree_2_colz = DatasetMismatchedJoin.aux_tree_name_2 + ".z"

default_value = ROOT.std.numeric_limits[int].min()

# Example 1: provide default values for all columns in case there was no
# match
display_1 = (
    df.DefaultValueFor(aux_tree_1_colidx, default_value)
    .DefaultValueFor(aux_tree_1_coly, default_value)
    .DefaultValueFor(aux_tree_2_colidx, default_value)
    .DefaultValueFor(aux_tree_2_colz, default_value)
    .Display(
        ("idx", aux_tree_1_colidx, aux_tree_2_colidx, "x", aux_tree_1_coly, aux_tree_2_colz))
)
display_1.Print()

In this scenario, we can also obtain a fine-grained processing of the matched datasets by composing operations on the first auxiliary dataset and operations on the second one. For example, we can decide to skip the entries where there is no match in the first auxiliary tree but fill with default values the entries in the second auxiliary tree in case there is still some mismatch.

**Note**: The `FilterAvailable` operation is a filter, so will take precedence w.r.t. running the `DefaultValueFor` in the event loop.

In [ ]:

# Example 2: skip the entire entry when there was no match for a column
# in the first auxiliary tree, but keep the entries when there is no match
# in the second auxiliary tree and provide a default value for those
display_2 = (
    df.DefaultValueFor(aux_tree_2_colidx, default_value)
    .DefaultValueFor(aux_tree_2_colz, default_value)
    .FilterAvailable(aux_tree_1_coly)
    .Display(
            ("idx", aux_tree_1_colidx, aux_tree_2_colidx, "x", aux_tree_1_coly, aux_tree_2_colz))
)
display_2.Print()

And finally, we can also use `FilterMissing` to keep entries that would be empty due to a mismatch in the join.

In [ ]:
# Example 3: Keep entries from the main tree for which there is no
# corresponding match in entries of the first auxiliary tree
display_3 = df.FilterMissing(aux_tree_1_colidx).Display(("idx", "x"))
display_3.Print()

In [ ]:
DatasetMismatchedJoin.cleanup_dataset()